In [1]:
import networkx as nx
import networkx.algorithms.community as nx_comm
from networkx.generators.community import LFR_benchmark_graph
from networkx.algorithms import bipartite
import numpy as np
import scipy as sp
from scipy.sparse import coo_array
from scipy import sparse
from cdlib import algorithms
from cdlib import evaluation
import sklearn
from utils import *
from distances import *
from consensus import *
import math
import itertools
import random
import time
from pathlib import Path

Note: to be able to use all crisp methods, you need to install some additional packages:  {'ASLPAw'}


In [2]:
cons_name = "v2"

In [54]:
# DeltaSOD is calculated following the paper titled "Integrating Microarray Data by Consensus Clustering"
# by Filkov and Skiena
# Assumes the elements of the cluster are named as 0-based indices
def v2_consensus(P_list, niter=10, starting_partition=None, verbose=False):
    G = nx.Graph(P_list[0]["graph"])
    n = len(list(G.nodes()))
    k = len(P_list)
    
    row = []
    col = []
    val = []
    for x in P_list:
        graph = x["graph"]
        partition = x["partition"]
        for cluster in partition:
            for i in range(len(cluster)):
                for j in range(i+1, len(cluster)):
                    item_1 = cluster[i]
                    item_2 = cluster[j]
                    row.append(min(int(item_1), int(item_2)))
                    col.append(max(int(item_1), int(item_2)))
                    val.append(int(1))
                    
    r = coo_array((val, (row, col)), shape=(n, n))
    r = r.tocsr()
    R = r.sum()
    if verbose:
        print("R:", R)
    
    rDense = r.toarray() # Should be upper triangular
    rDense = rDense + rDense.T # Making symmetric
    
    A = nx.to_numpy_array(G)
    #print(A.shape)
    nz_rows, nz_cols = np.nonzero(A)
    Aw = np.take(rDense, np.where(A > 0))
    Aw = np.zeros(A.shape)
    nz_elems = []
    for i in range(len(nz_rows)):
        if rDense[nz_rows[i], nz_cols[i]] > (0.5 * k):
            nz_elems.append((nz_rows[i], nz_cols[i], rDense[nz_rows[i], nz_cols[i]]))
            Aw[nz_rows[i], nz_cols[i]] = rDense[nz_rows[i], nz_cols[i]]
    nz_elems = sorted(nz_elems, key=lambda x: x[2], reverse=True)
    #print(len(nz_elems))
    
    K = k - 2 * rDense
    np.fill_diagonal(K, -k) # Adding diagonal entries
    
    refined_partition = None
    if starting_partition:
        refined_partition = list(starting_partition)
    else:
        refined_partition = []
        for i in range(n):
            refined_partition.append([str(i)])
    
    refined_partition_map = clust_lst_to_map(refined_partition)
    
    items = list(refined_partition_map.keys())
    M = np.zeros((n, len(refined_partition)))
    for item_1 in items:
        for partition_id in range(len(refined_partition)):
            for item_2 in refined_partition[partition_id]:
                if(item_1 != item_2):
                    M[int(item_1),partition_id] = M[int(item_1),partition_id] + K[int(item_1), int(item_2)]
    
    mv = np.min(M, axis=1)
    mb = np.argmin(M, axis=1)
    
    count = 0
    it = 1
    while(it <= niter):
        opt_item = items[0]
        opt_deltaS = 0
        opt_a = refined_partition_map[items[0]]
        opt_b = refined_partition_map[items[0]]
        opt_x = int(opt_item)
        flag = False
        for i in range(len(nz_elems)):
            if verbose:
                print("nz_elems[", i, "]", nz_elems[i])
            x1 = nz_elems[i][0]
            x2 = nz_elems[i][1]
            a1 = refined_partition_map[str(x1)]
            a2 = refined_partition_map[str(x2)]
            deltaS1 = M[x1,a2] - M[x1,a1]
            deltaS2 = M[x2,a1] - M[x2,a2]
            x = None
            item = None
            b = None
            deltaS = None
            if (deltaS1 < deltaS2):
                x = x1
                item = str(x1)
                a = a1
                b = a2
                deltaS = deltaS1
            else:
                x = x2
                item = str(x2)
                a = a2
                b = a1
                deltaS = deltaS2
            if (deltaS is not None) and (deltaS < 0) and (a != b):
                opt_item = item
                opt_deltaS = deltaS
                opt_a = a
                opt_b = b
                opt_x = x
                if verbose:
                    print("---")
                    print("Move Count:", count+1, "Optimum move results in", opt_deltaS)
                    print("Move:", opt_item)
                    print("From", opt_a, ":", refined_partition[opt_a])
                    print("To", opt_b, ":", refined_partition[opt_b])
                for item in items:
                    y = int(item)
                    if y != opt_x:
                        M[y, opt_a] = M[y, opt_a] - K[y, opt_x]
                        M[y, opt_b] = M[y, opt_b] + K[y, opt_x]
            
                mv = np.min(M, axis=1)
                mb = np.argmin(M, axis=1)
            
                refined_partition[opt_a].remove(opt_item)
                refined_partition[opt_b].append(opt_item)
                refined_partition_map[opt_item] = opt_b
                if verbose:
                    print("---")
            
                count = count + 1
                flag = True
        if flag == False:
            break

        it = it + 1
    
    empty_clusters = []
    for i in range(len(refined_partition)):
        if len(refined_partition[i]) == 0:
            empty_clusters.append(i)
            
    empty_clusters.sort(reverse=True)
    for e in empty_clusters:
        del refined_partition[e]
        
    G = nx.from_scipy_sparse_array(r)
    
    return {"graph": nx.Graph(G), "partition": list(refined_partition)}

# n=200

In [4]:
n = 200
expected_clusters = []
for i in range(4):
    expected_clusters.append(random.randint(int(n ** (1. / 3)),3*int(n ** (1. / 2))))
    
alg_params = {
    "label_propagation": None,
    "leiden": None,
    "significance_communities": None,
    "surprise_communities": None,
    "greedy_modularity": None,
    "paris": None,
    "louvain": {
        "resolution": [0.75, 1.0, 1.25, 1.5],
        "randomize": [314159, 2718]
    },
    "infomap": None,
    "walktrap": None,
    "markov_clustering": {
        "inflation": [1.2, 1.5, 2, 2.5],
        "pruning_threshold": [0.01, 0.001],
        "convergence_check_frequency": [100]
    },
    "em": {
        "k": list(expected_clusters)
    },
    "sbm_dl": None,
    "spinglass": {
        "spins": list(expected_clusters)
    },
    "ricci_community": {
        "alpha": [0.3, 0.5, 0.6, 0.75]
    }
}

clustering_enumeration = []
count = 0
for alg, params in alg_params.items():
    param_combinations = []
    param_names = []
    if params is not None:
        iterables = []
        param_names = []
        for param in params.keys():
            iterables.append(list(params[param]))
            param_names.append(param)
        param_combinations = list(itertools.product(*iterables))
    if len(param_combinations) > 0:
        for param_combination in param_combinations:
            expr = "algorithms."+alg+"(G"
            for i in range(len(param_names)):
                expr = expr + "," + param_names[i] + "=" + str(param_combination[i])
            expr = expr + ")"
            clustering_enumeration.append((expr,count))
            count = count + 1      
    else:
        expr = "algorithms."+alg+"(G)"
        clustering_enumeration.append((expr,count))
        count = count + 1
        
print(clustering_enumeration)

[('algorithms.label_propagation(G)', 0), ('algorithms.leiden(G)', 1), ('algorithms.significance_communities(G)', 2), ('algorithms.surprise_communities(G)', 3), ('algorithms.greedy_modularity(G)', 4), ('algorithms.paris(G)', 5), ('algorithms.louvain(G,resolution=0.75,randomize=314159)', 6), ('algorithms.louvain(G,resolution=0.75,randomize=2718)', 7), ('algorithms.louvain(G,resolution=1.0,randomize=314159)', 8), ('algorithms.louvain(G,resolution=1.0,randomize=2718)', 9), ('algorithms.louvain(G,resolution=1.25,randomize=314159)', 10), ('algorithms.louvain(G,resolution=1.25,randomize=2718)', 11), ('algorithms.louvain(G,resolution=1.5,randomize=314159)', 12), ('algorithms.louvain(G,resolution=1.5,randomize=2718)', 13), ('algorithms.infomap(G)', 14), ('algorithms.walktrap(G)', 15), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.01,convergence_check_frequency=100)', 16), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.001,convergence_check_frequency=100)

In [57]:
import random

n = 200
fileprefix = "LFR/" + "n" + str(n) + "/"
mus = [1, 2, 3, 4]
#mus = [4]
gammas = [30]
betas = [11]
for mu in mus:
    for gamma in gammas:
        for beta in betas:
            P_list = []
            fname = "LFR_n" + str(n) + "_mu0" + str(mu) + "_gamma" + str(gamma) + "_beta" + str(beta)
            graph_file = fileprefix + fname + ".mtx"
            print(graph_file)
            G = None
            with open(graph_file) as f:
                G = nx.from_scipy_sparse_array(spio.mmread(f), create_using=nx.Graph)
                coms = None
                for k in clustering_enumeration:
                    clust_file = fileprefix + fname + "." + str(k[1])
                    if Path(clust_file).is_file():
                        partition = read_clust_lst(clust_file)
                        P_list.append({"graph": nx.Graph(G), "partition": list(partition)})
                t1 = time.time()
                P_star = v2_consensus(P_list, niter=1000, starting_partition = None, verbose=False)
                t2 = time.time()
                print("mu", mu, ", number of clusters", len(P_star["partition"]))
                print("Time:", t2-t1)
                write_clust_lst(P_star["partition"], fileprefix + fname + "." + cons_name)

LFR/n200/LFR_n200_mu01_gamma30_beta11.mtx
mu 1 , number of clusters 16
Time: 0.4114687442779541
LFR/n200/LFR_n200_mu02_gamma30_beta11.mtx
mu 2 , number of clusters 16
Time: 0.4053184986114502
LFR/n200/LFR_n200_mu03_gamma30_beta11.mtx
mu 3 , number of clusters 17
Time: 0.4519343376159668
LFR/n200/LFR_n200_mu04_gamma30_beta11.mtx
mu 4 , number of clusters 24
Time: 0.5511479377746582


# n=1000

In [58]:
n = 1000
expected_clusters = []
for i in range(4):
    expected_clusters.append(random.randint(int(n ** (1. / 3)),3*int(n ** (1. / 2))))
    
alg_params = {
    "label_propagation": None,
    "leiden": None,
    "significance_communities": None,
    "surprise_communities": None,
    "greedy_modularity": None,
    "paris": None,
    "louvain": {
        "resolution": [0.75, 1.0, 1.25, 1.5],
        "randomize": [314159, 2718]
    },
    "infomap": None,
    "walktrap": None,
    "markov_clustering": {
        "inflation": [1.2, 1.5, 2, 2.5],
        "pruning_threshold": [0.01, 0.001],
        "convergence_check_frequency": [100]
    },
    "em": {
        "k": list(expected_clusters)
    },
    "sbm_dl": None,
    "spinglass": {
        "spins": list(expected_clusters)
    },
    "ricci_community": {
        "alpha": [0.3, 0.5, 0.6, 0.75]
    }
}

clustering_enumeration = []
count = 0
for alg, params in alg_params.items():
    param_combinations = []
    param_names = []
    if params is not None:
        iterables = []
        param_names = []
        for param in params.keys():
            iterables.append(list(params[param]))
            param_names.append(param)
        param_combinations = list(itertools.product(*iterables))
    if len(param_combinations) > 0:
        for param_combination in param_combinations:
            expr = "algorithms."+alg+"(G"
            for i in range(len(param_names)):
                expr = expr + "," + param_names[i] + "=" + str(param_combination[i])
            expr = expr + ")"
            clustering_enumeration.append((expr,count))
            count = count + 1      
    else:
        expr = "algorithms."+alg+"(G)"
        clustering_enumeration.append((expr,count))
        count = count + 1
        
print(clustering_enumeration)

[('algorithms.label_propagation(G)', 0), ('algorithms.leiden(G)', 1), ('algorithms.significance_communities(G)', 2), ('algorithms.surprise_communities(G)', 3), ('algorithms.greedy_modularity(G)', 4), ('algorithms.paris(G)', 5), ('algorithms.louvain(G,resolution=0.75,randomize=314159)', 6), ('algorithms.louvain(G,resolution=0.75,randomize=2718)', 7), ('algorithms.louvain(G,resolution=1.0,randomize=314159)', 8), ('algorithms.louvain(G,resolution=1.0,randomize=2718)', 9), ('algorithms.louvain(G,resolution=1.25,randomize=314159)', 10), ('algorithms.louvain(G,resolution=1.25,randomize=2718)', 11), ('algorithms.louvain(G,resolution=1.5,randomize=314159)', 12), ('algorithms.louvain(G,resolution=1.5,randomize=2718)', 13), ('algorithms.infomap(G)', 14), ('algorithms.walktrap(G)', 15), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.01,convergence_check_frequency=100)', 16), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.001,convergence_check_frequency=100)

In [59]:
import random

n = 1000
fileprefix = "LFR/" + "n" + str(n) + "/"
mus = [1, 2, 3, 4]
#mus = [4]
gammas = [30]
betas = [11]
for mu in mus:
    for gamma in gammas:
        for beta in betas:
            P_list = []
            fname = "LFR_n" + str(n) + "_mu0" + str(mu) + "_gamma" + str(gamma) + "_beta" + str(beta)
            graph_file = fileprefix + fname + ".mtx"
            print(graph_file)
            G = None
            with open(graph_file) as f:
                G = nx.from_scipy_sparse_array(spio.mmread(f), create_using=nx.Graph)
                coms = None
                for k in clustering_enumeration:
                    clust_file = fileprefix + fname + "." + str(k[1])
                    if Path(clust_file).is_file():
                        partition = read_clust_lst(clust_file)
                        P_list.append({"graph": nx.Graph(G), "partition": list(partition)})
                t1 = time.time()
                P_star = v2_consensus(P_list, niter=1000, starting_partition=None, verbose=False)
                t2 = time.time()
                print("mu", mu, ", number of clusters", len(P_star["partition"]))
                print("Time:", t2-t1)
                write_clust_lst(P_star["partition"], fileprefix + fname + "." + cons_name)
                

LFR/n1000/LFR_n1000_mu01_gamma30_beta11.mtx
mu 1 , number of clusters 39
Time: 6.196271657943726
LFR/n1000/LFR_n1000_mu02_gamma30_beta11.mtx
mu 2 , number of clusters 38
Time: 7.985594272613525
LFR/n1000/LFR_n1000_mu03_gamma30_beta11.mtx
mu 3 , number of clusters 41
Time: 9.850877285003662
LFR/n1000/LFR_n1000_mu04_gamma30_beta11.mtx
mu 4 , number of clusters 61
Time: 11.924928188323975


# n=5000

In [60]:
n = 5000
expected_clusters = []
for i in range(4):
    expected_clusters.append(random.randint(int(n ** (1. / 3)),3*int(n ** (1. / 2))))
    
alg_params = {
    "label_propagation": None,
    "leiden": None,
    "significance_communities": None,
    "surprise_communities": None,
    "greedy_modularity": None,
    "paris": None,
    "louvain": {
        "resolution": [0.75, 1.0, 1.25, 1.5],
        "randomize": [314159, 2718]
    },
    "infomap": None,
    "walktrap": None,
    "markov_clustering": {
        "inflation": [1.2, 1.5, 2, 2.5],
        "pruning_threshold": [0.01, 0.001],
        "convergence_check_frequency": [100]
    },
    "em": {
        "k": list(expected_clusters)
    },
    "sbm_dl": None,
    "spinglass": {
        "spins": list(expected_clusters)
    },
    "ricci_community": {
        "alpha": [0.3, 0.5, 0.6, 0.75]
    }
}

clustering_enumeration = []
count = 0
for alg, params in alg_params.items():
    param_combinations = []
    param_names = []
    if params is not None:
        iterables = []
        param_names = []
        for param in params.keys():
            iterables.append(list(params[param]))
            param_names.append(param)
        param_combinations = list(itertools.product(*iterables))
    if len(param_combinations) > 0:
        for param_combination in param_combinations:
            expr = "algorithms."+alg+"(G"
            for i in range(len(param_names)):
                expr = expr + "," + param_names[i] + "=" + str(param_combination[i])
            expr = expr + ")"
            clustering_enumeration.append((expr,count))
            count = count + 1      
    else:
        expr = "algorithms."+alg+"(G)"
        clustering_enumeration.append((expr,count))
        count = count + 1
        
print(clustering_enumeration)

[('algorithms.label_propagation(G)', 0), ('algorithms.leiden(G)', 1), ('algorithms.significance_communities(G)', 2), ('algorithms.surprise_communities(G)', 3), ('algorithms.greedy_modularity(G)', 4), ('algorithms.paris(G)', 5), ('algorithms.louvain(G,resolution=0.75,randomize=314159)', 6), ('algorithms.louvain(G,resolution=0.75,randomize=2718)', 7), ('algorithms.louvain(G,resolution=1.0,randomize=314159)', 8), ('algorithms.louvain(G,resolution=1.0,randomize=2718)', 9), ('algorithms.louvain(G,resolution=1.25,randomize=314159)', 10), ('algorithms.louvain(G,resolution=1.25,randomize=2718)', 11), ('algorithms.louvain(G,resolution=1.5,randomize=314159)', 12), ('algorithms.louvain(G,resolution=1.5,randomize=2718)', 13), ('algorithms.infomap(G)', 14), ('algorithms.walktrap(G)', 15), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.01,convergence_check_frequency=100)', 16), ('algorithms.markov_clustering(G,inflation=1.2,pruning_threshold=0.001,convergence_check_frequency=100)

In [62]:
import random

n = 5000
fileprefix = "LFR/" + "n" + str(n) + "/"
mus = [1, 2, 3, 4]
#mus = [4]
gammas = [30]
betas = [11]
for mu in mus:
    for gamma in gammas:
        for beta in betas:
            P_list = []
            fname = "LFR_n" + str(n) + "_mu0" + str(mu) + "_gamma" + str(gamma) + "_beta" + str(beta)
            graph_file = fileprefix + fname + ".mtx"
            print(graph_file)
            G = None
            with open(graph_file) as f:
                G = nx.from_scipy_sparse_array(spio.mmread(f), create_using=nx.Graph)
                coms = None
                for k in clustering_enumeration:
                    clust_file = fileprefix + fname + "." + str(k[1])
                    if Path(clust_file).is_file():
                        partition = read_clust_lst(clust_file)
                        P_list.append({"graph": nx.Graph(G), "partition": list(partition)})
                t1 = time.time()
                P_star = v2_consensus(P_list, niter=1000, starting_partition = None, verbose=False)
                t2 = time.time()
                print("mu", mu, ", number of clusters", len(P_star["partition"]))
                print("Time:", t2-t1)
                write_clust_lst(P_star["partition"], fileprefix + fname + "." + cons_name)
                

LFR/n5000/LFR_n5000_mu01_gamma30_beta11.mtx
mu 1 , number of clusters 203
Time: 255.52839589118958
LFR/n5000/LFR_n5000_mu02_gamma30_beta11.mtx
mu 2 , number of clusters 201
Time: 270.2532675266266
LFR/n5000/LFR_n5000_mu03_gamma30_beta11.mtx
mu 3 , number of clusters 214
Time: 306.2652680873871
LFR/n5000/LFR_n5000_mu04_gamma30_beta11.mtx
mu 4 , number of clusters 385
Time: 343.4427273273468
